<a href="https://colab.research.google.com/github/CodeHunterOfficial/ABC_DataMining/blob/main/TimeSeries/TimeSeries-2025/New/%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B0%D1%8F_%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0_%E2%84%96%E2%80%AF3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **Практическая работа № 3**  
## **Сравнительный анализ классических моделей машинного обучения для анализа и прогнозирования временных рядов (табличные данные)**

---

### **1. Цель и задачи работы**

**Цель работы** — формирование у обучающегося системного представления о методологии сравнительного анализа классических моделей машинного обучения, применяемых к задачам прогнозирования временных рядов на основе табличных данных, при условии соблюдения строгих требований к предобработке, временной валидации, настройке гиперпараметров и интерпретации результатов. Работа направлена на освоение практических навыков трансформации временного ряда в регрессионную задачу, корректной оптимизации моделей с использованием временной кросс-валидации, диагностики остатков, статистического сравнения и публикации воспроизводимых результатов в соответствии с передовыми научно-методическими стандартами.

**Основные задачи работы:**  
1. Использовать обработанный табличный датасет из Практической работы № 1, содержащий не менее 720 последовательных наблюдений, с подтверждённой стационарностью, стабилизированной дисперсией и богатым набором инженерных признаков (лаги, скользящие окна, циклические кодировки).  
2. Формализовать задачу прогнозирования как задачу табличной регрессии с соблюдением временного порядка и отсутствием утечки будущего.  
3. Определить пространства гиперпараметров для всех моделей и провести их **настройку с использованием временной кросс-валидации** (`TimeSeriesSplit`) через `GridSearchCV` и `RandomizedSearchCV`.  
4. Реализовать и сравнить три стратегии многопшагового прогнозирования: рекурсивную, прямую и гибридную — для горизонтов \( h = 1, 7, 30 \).  
5. Построить и протестировать классические модели машинного обучения:  
   - Линейные: OLS, Ridge, Lasso, ElasticNet  
   - KNNRegressor  
   - Decision Tree, Random Forest, Extra Trees  
   - Градиентный бустинг: XGBoost, LightGBM, CatBoost  
   - Support Vector Regression: SVR, NuSVR, LinearSVR  
   - MLPRegressor (многослойный перцептрон)  
   - Gaussian Process Regressor  
   - Робастные регрессоры: Huber, Theil–Sen, RANSAC  
   - А также бенчмарки: Naive, Seasonal Naive  
6. Провести углублённую диагностику адекватности моделей: тест Льюнга–Бокса на автокорреляцию остатков, анализ гомоскедастичности, проверку нормальности распределения ошибок (Q-Q график, тест Шапиро–Уилка).  
7. Оценить качество прогнозов по метрикам: MAE, RMSE, MAPE, MASE, SMAPE, R², RMSLE — **на независимой тестовой выборке, не участвовавшей в настройке**.  
8. Применить тест Дикболда–Мариано для статистического сравнения точности прогнозов.  
9. Проанализировать интерпретируемость моделей: коэффициенты линейных моделей, важность признаков, SHAP-объяснения, устойчивость выводов при разных разбиениях.  
10. Оценить вычислительную сложность: время настройки, прогнозирования и потребление памяти.  
11. Разработать интерактивный веб-инструмент для визуального и функционального сравнения моделей, включая отображение гиперпараметров, важности признаков и SHAP-диаграмм.  
12. Обеспечить воспроизводимость и открытость результатов: опубликовать код, сравнительные таблицы, веб-приложение и аналитический отчёт.

---

### **2. Теоретические предпосылки**

Модели машинного обучения не учитывают временную структуру данных по умолчанию, что делает их применение к временным рядам методологически нетривиальной задачей. Успешное использование ML-подходов требует **трансформации ряда в табличную форму** с помощью лагов, скользящих статистик и циклических кодировок, а также **строгого соблюдения временного порядка** при разделении и валидации.

Ключевым условием объективного сравнения является **настройка гиперпараметров каждой модели на равных основаниях**. Использование параметров по умолчанию приводит к смещённым выводам, так как разные алгоритмы чувствительны к гиперпараметрам в разной степени. Однако для временных рядов стандартная кросс-валидация с перемешиванием (shuffle=True) недопустима — она нарушает причинно-временную структуру и приводит к утечке будущего. Единственно корректный подход — временная кросс-валидация (rolling origin, expanding window), реализуемая в scikit-learn через TimeSeriesSplit.



Выбор метрики для оптимизации также критичен: предпочтение отдаётся **масштаб-инвариантным и устойчивым к выбросам метрикам**, таким как MASE или MAE, особенно при наличии аномалий.

После настройки модель оценивается **только на независимой тестовой выборке**, что гарантирует объективность итоговых метрик.

Интерпретируемость рассматривается как неотъемлемая часть анализа. Для линейных моделей это — коэффициенты и их статистическая значимость; для ансамблей — важность признаков и SHAP-значения. Это позволяет не только прогнозировать, но и **объяснять динамику системы**, выявлять ключевые детерминанты и проверять соответствие модели предметной области.

---

### **3. Порядок выполнения работы**

Работа выполняется в несколько последовательных этапов, каждый из которых направлен на достижение конкретных образовательных и исследовательских результатов.

#### **3.1. Использование данных из Практической работы № 1**

**Задача:** Обеспечить методологическую преемственность и воспроизводимость.  
**Требования к выполнению:**  
– В качестве входных данных используется финальный обработанный датасет из Практической работы № 1:  
  • Объём: не менее 720 последовательных наблюдений;  
  • Наличие целевой переменной и не менее пяти дополнительных признаков (включая лаги \( y_{t-1}, y_{t-7}, y_{t-30} \), скользящие средние, циклические кодировки времени);  
  • Подтверждённая стационарность (результаты ADF/KPSS);  
  • Сохранённые параметры преобразований (например, \( \lambda \) для Бокса–Кокса или параметры лог-трансформации).  
– Все признаки представлены в табличной форме без пропусков на момент обучения.  
– Обратное преобразование прогнозов осуществляется с использованием сохранённых параметров.

#### **3.2. Формализация задачи и разделение данных**

**Задача:** Корректно сформулировать задачу машинного обучения с учётом временной структуры.  
**Требования к выполнению:**  
– Целевая переменная: \( y_t \); признаки \( X_t \): все доступные лаги, статистики и экзогенные факторы.  
– Разделение данных:  
  • Тренировочная + валидационная часть: всё до \( T - 90 \);  
  • Финальный тест: последние 90 наблюдений — **только для итоговой оценки, не участвует в настройке**.  
– Для кросс-валидации: `TimeSeriesSplit(n_splits=5, test_size=60)` — без перемешивания, с сохранением порядка.

#### **3.3. Определение пространств гиперпараметров и настройка моделей**

**Задача:** Привести каждую модель к её оптимальной конфигурации с использованием временной кросс-валидации.  
**Требования к выполнению:**  
– Для каждой модели определяется пространство гиперпараметров на основе рекомендаций литературы:  
  • **Линейные модели**: `alphas = np.logspace(-4, 4, 50)` для Ridge/Lasso;  
  • **Деревья**: `max_depth ∈ [5,10,15,None]`, `n_estimators ∈ [100,200,300]`;  
  • **XGBoost/LightGBM**: `learning_rate ∈ [0.01,0.3]`, `max_depth ∈ [3,10]`, `subsample ∈ [0.6,1.0]`;  
  • **SVR**: `C ∈ [0.1,1,10,100]`, `gamma ∈ ['scale','auto',0.001,0.01,0.1]`;  
  • **MLP**: `hidden_layer_sizes ∈ [(50,), (100,50), (50,50,50)]`, `alpha ∈ [1e-4,1e-2]`.  
– Настройка:  
  • `GridSearchCV` — для моделей с небольшим числом гиперпараметров (линейные, SVR, KNN);  
  • `RandomizedSearchCV(n_iter=30)` — для моделей с большим пространством (бустинг, MLP);  
  • Целевая метрика: **MASE** или **MAE** (масштаб-инвариантные);  
  • Все процедуры выполняются **только на тренировочной части** с `cv=TimeSeriesSplit`.  
– Фиксация: оптимальные гиперпараметры, значение метрики на валидации, время настройки.

#### **3.4. Стратегии многопшагового прогнозирования**

**Задача:** Реализовать и сравнить подходы к прогнозу на горизонт \( h \geq 7 \).  
**Требования к выполнению:**  
– **Рекурсивная стратегия:** одна настроенная модель; прогноз на \( t+1 \) используется как вход для \( t+2 \);  
– **Прямая стратегия:** отдельная настройка и обучение модели для каждого шага \( t+1, \dots, t+h \);  
– **Гибридная стратегия:** например, рекурсивная для \( h \leq 3 \), прямая — для \( h > 3 \);  
– Для каждой стратегии и каждого горизонта (\( h = 1, 7, 30 \)):  
  • Расчёт MAE, RMSE на каждом шаге;  
  • Фиксация общего времени вычислений (включая настройку для прямой стратегии);  
  • Построение графика накопления ошибки и времени.

#### **3.5. Построение и настройка моделей**

**Задача:** Реализовать единообразный интерфейс для всех классических ML-моделей.  
**Требования к выполнению:**  
– **Линейные модели:**  
  • OLS: `LinearRegression()`;  
  • Ridge/Lasso/ElasticNet: с кросс-валидацией по α;  
– **KNN:** `KNeighborsRegressor(n_neighbors=5, weights='distance')` с настройкой `n_neighbors` и `weights`;  
– **Деревья:**  
  • `DecisionTreeRegressor(max_depth=10)`;  
  • `RandomForestRegressor(n_estimators=200)`;  
  • `ExtraTreesRegressor(n_estimators=200)`;  
– **Градиентный бустинг:**  
  • `XGBRegressor(n_estimators=200, objective='reg:squarederror')`;  
  • `LGBMRegressor(n_estimators=200)`;  
  • `CatBoostRegressor(iterations=200, verbose=False)`;  
– **Метод опорных векторов:**  
  • `SVR(kernel='rbf')`, `NuSVR()`, `LinearSVR(max_iter=10000)`;  
– **Нейросети:** `MLPRegressor(hidden_layer_sizes=(100,50), max_iter=500)`;  
– **Гауссовские процессы:** `GaussianProcessRegressor(alpha=1e-5)`;  
– **Робастные регрессоры:** `HuberRegressor()`, `TheilSenRegressor()`, `RANSACRegressor()`;  
– **Бенчмарки:** Naive (\( \hat{y}_{t+h} = y_t \)), Seasonal Naive (\( \hat{y}_{t+h} = y_{t+h-s} \));  
– Для всех моделей: прогноз на тесте, обратное преобразование, сохранение времени и метаданных.

#### **3.6. Диагностика адекватности моделей**

**Задача:** Убедиться, что модель корректно аппроксимирует динамику ряда.  
**Требования к выполнению:**  
– **Тест Льюнга–Бокса** на остатках с использованием `statsmodels.stats.diagnostic.acorr_ljungbox` (H₀: автокорреляция отсутствует; p > 0.05 — адекватность);  
– **Гомоскедастичность:** визуальный анализ графика остатков \( \varepsilon_t \) против прогнозов \( \hat{y}_t \);  
– **Нормальность остатков:**  
  • Q-Q график;  
  • Тест Шапиро–Уилка (`scipy.stats.shapiro`); p > 0.05 — нормальность не отвергается;  
– **ACF/PACF остатков** — должны лежать в пределах доверительного интервала.  
– Диагностика обязательна для всех линейных моделей и топ-5 моделей по метрикам.

#### **3.7. Оценка качества и статистическое сравнение**

**Задача:** Обоснованно сравнить модели по объективным критериям.  
**Требования к выполнению:**  
– Расчёт метрик **на финальной тестовой выборке** для каждого горизонта: MAE, RMSE, MAPE, MASE, SMAPE, R², RMSLE;  
– Применение **теста Дикболда–Мариано** (`statsmodels.stats.diagnostic.dm_test`) для попарного сравнения прогнозов топ-5 моделей;  
– Составление **сравнительных таблиц**:  
  • Модели × стратегии × горизонты (метрики, время настройки, время прогноза);  
  • Интерпретируемость (коэффициенты, SHAP, важность признаков);  
  • Диагностические p-значения (Ljung–Box, Shapiro–Wilk);  
– Ранжирование моделей по взвешенной оценке, учитывающей точность (50%), адекватность (30%) и интерпретируемость (20%).

#### **3.8. Анализ интерпретируемости моделей**

**Задача:** Оценить способность модели объяснять динамику системы.  
**Требования к выполнению:**  
– Для **линейных моделей**: анализ коэффициентов, их знаков, величин и p-значений (через `statsmodels`);  
– Для **ансамблей деревьев**: встроенная важность признаков и **SHAP-значения** (`shap.TreeExplainer`);  
– Для **KNN, SVR, MLP**: интерпретируемость ограничена — фиксируется как ограничение;  
– Проведение **локальной интерпретации**: SHAP-водопадные диаграммы для отдельных точек прогноза;  
– Анализ **устойчивости**: повторный запуск настройки с разными random_state — насколько стабильны ключевые признаки?  
– Результаты интерпретации включаются в сравнительные таблицы и веб-интерфейс.

#### **3.9. Анализ вычислительной эффективности**

**Задача:** Оценить практическую применимость моделей в условиях ограниченных ресурсов.  
**Требования к выполнению:**  
– Фиксация времени:  
  • Настройка (в секундах),  
  • Прогнозирование (в миллисекундах на точку);  
– Оценка потребления памяти (опционально, через `memory_profiler`);  
– Сравнение `GridSearchCV` и `RandomizedSearchCV` по качеству/времени;  
– Включение столбца «Вычислительная сложность» в сравнительные таблицы: низкая/средняя/высокая.

#### **3.10. Разработка веб-интерфейса для анализа результатов**

**Задача:** Создать интерактивное приложение для визуального и функционального сравнения моделей.  
**Требования к выполнению:**  
– Возможность загрузки пользовательского CSV или выбора встроенного датасета (из работы № 1);  
– Выбор модели, горизонта прогноза и стратегии;  
– Автоматическое построение:  
  • Графика прогноза с остатками;  
  • Важности признаков (столбчатая диаграмма);  
  • SHAP-диаграмм (глобальных и локальных);  
  • Таблицы метрик, гиперпараметров и времени;  
– Экспорт прогноза и отчёта в CSV/PDF;  
– Рекомендуемые технологии: Streamlit, Plotly, scikit-learn, shap, xgboost.

#### **3.11. Публикация результатов исследования**

**Задача:** Обеспечить открытость и воспроизводимость работы.  
**Требования к выполнению:**  
– Для кода и результатов составляется метаописание, включающее:  
  • Список моделей и пространств гиперпараметров;  
  • Оптимальные гиперпараметры и метрики на валидации;  
  • Сравнительные таблицы по всем моделям, стратегиям и горизонтам;  
  • Ключевые выводы и рекомендации;  
  • Инструкцию по запуску веб-приложения;  
– Все материалы публикуются в публичном репозитории (GitHub/GitLab);  
– Веб-приложение развёртывается на Hugging Face Spaces, Render или Railway;  
– Использование открытых лицензий: MIT (код), CC BY 4.0 (данные).

---

### **4. Дополнительные исследовательские задания**

1. Сравнение эффективности `GridSearchCV` и `RandomizedSearchCV` по качеству и времени настройки.  
2. Анализ влияния масштабирования признаков (StandardScaler, RobustScaler) на качество нелинейных моделей.  
3. Построение ансамбля из топ-3 ML-моделей по каждому горизонту (простое усреднение или стекинг).  
4. Оценка устойчивости SHAP-значений при разных разбиениях данных.  
5. Анализ чувствительности прогнозов к выбросам в обучающей выборке (сравнение робастных и обычных моделей).  
6. Исследование зависимости времени настройки от длины временного ряда (масштабируемость).

---

### **5. Требования к отчёту**

Отчёт оформляется в соответствии с ГОСТ 7.32–2017 и должен содержать следующие разделы:  
1. Введение — постановка задачи, актуальность ML для временных рядов, обзор литературы, указание на преемственность с Практической работой № 1.  
2. Методология — описание данных, трансформация ряда в признаки, пространства гиперпараметров, стратегии прогнозирования, список моделей и инструментов.  
3. Результаты экспериментов — сравнительные таблицы, графики ошибок, результаты теста Дикболда–Мариано, диагностика остатков, SHAP-визуализации.  
4. Обсуждение — интерпретация важных признаков, анализ влияния настройки гиперпараметров, сравнение стратегий, вычислительная эффективность, соответствие выводов предметной области.  
5. Заключение — итоговые выводы, практические рекомендации по выбору модели в продакшене, ограничения исследования.  
6. Список использованных источников — оформлен в соответствии с ГОСТ Р 7.0.5–2008.  
7. Приложения — фрагменты кода, скриншоты веб-интерфейса, примеры сравнительных таблиц, таблицы гиперпараметров.  
Отчёт сопровождается ссылками на репозиторий с кодом, веб-приложение и опубликованные данные.

---

### **6. Критерии оценивания**

| Оценка | Критерии |
|--------|----------|
| **Отлично** | Полное выполнение всех этапов; настройка гиперпараметров через временную кросс-валидацию для всех моделей; реализация всех трёх стратегий прогнозирования; углублённая диагностика остатков; тест Дикболда–Мариано; развёрнутый анализ интерпретируемости (SHAP, коэффициенты, устойчивость); оценка вычислительной сложности; функциональный веб-интерфейс с отображением гиперпараметров и объяснений; публикация всех материалов; отчёт, соответствующий ГОСТ. |
| **Хорошо** | Выполнение основных этапов; настройка большинства моделей; наличие сравнительных таблиц и базовой диагностики; реализация основных моделей и стратегий; отсутствие лишь отдельных элементов (например, SHAP или анализа времени). |
| **Удовлетворительно** | Реализованы 6–8 моделей без полной настройки; есть сравнение по метрикам; отсутствует диагностика остатков, интерпретируемость или стратегии прогнозирования. |
| **Неудовлетворительно** | Отсутствуют ключевые компоненты работы (данные из работы № 1 не использованы, отсутствует настройка гиперпараметров, нет сравнительного анализа, отчёт не представлен). |
